Import Libraries and Load Config

In [1]:
import psycopg2
import pandas as pd
import configparser


Load Redshift Config from dwh.cfg

In [2]:
# Load config
config = configparser.ConfigParser()
config.read('dwh.cfg')

# Set connection parameters
HOST = config.get('CLUSTER', 'HOST')
DB_NAME = config.get('CLUSTER', 'DB_NAME')
DB_USER = config.get('CLUSTER', 'DB_USER')
DB_PASSWORD = config.get('CLUSTER', 'DB_PASSWORD')
DB_PORT = config.get('CLUSTER', 'DB_PORT')


Connect to Redshift

In [3]:
# Connect to Redshift
conn = psycopg2.connect(
    host=HOST,
    dbname=DB_NAME,
    user=DB_USER,
    password=DB_PASSWORD,
    port=DB_PORT
)
cur = conn.cursor()


Query Helper Function

In [4]:
# Helper function to run a query and return a DataFrame
def run_query(query):
    cur.execute(query)
    df = pd.DataFrame(cur.fetchall(), columns=[desc[0] for desc in cur.description])
    return df


Check staging table counts

In [17]:
run_query("""
SELECT 'staging_events' AS table_name, COUNT(*) AS row_count FROM staging_events
UNION ALL
SELECT 'staging_songs', COUNT(*) FROM staging_songs;
""")


,table_name,row_count
0,staging_events,8056
1,staging_songs,82


Check dimension/fact tables

In [18]:
run_query("""
SELECT 'users' AS table_name, COUNT(*) FROM users
UNION ALL
SELECT 'songs', COUNT(*) FROM songs
UNION ALL
SELECT 'artists', COUNT(*) FROM artists
UNION ALL
SELECT 'time', COUNT(*) FROM time
UNION ALL
SELECT 'songplays', COUNT(*) FROM songplays;
""")


,table_name,count
0,users,105
1,songs,71
2,artists,69
3,time,6813
4,songplays,6820


Sample songplays

In [19]:
run_query("SELECT * FROM songplays LIMIT 5;")


,songplay_id,start_time,user_id,level,song_id,artist_id,session_id,location,user_agent
0,0,2018-11-01 21:05:52,8,free,None,None,139,"Phoenix-Mesa-Scottsdale, AZ","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK..."
1,6,2018-11-01 21:42:00,10,free,None,None,9,"Washington-Arlington-Alexandria, DC-VA-MD-WV","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."
2,8,2018-11-01 21:52:05,26,free,None,None,169,"San Jose-Sunnyvale-Santa Clara, CA","""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5..."
3,12,2018-11-01 22:23:14,101,free,None,None,100,"New Orleans-Metairie, LA","""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK..."
4,14,2018-11-03 01:05:50,53,free,None,None,52,"Klamath Falls, OR","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK..."


Most active users

In [20]:
run_query("""
SELECT user_id, COUNT(*) AS total_plays
FROM songplays
GROUP BY user_id
ORDER BY total_plays DESC
LIMIT 5;
""")

,user_id,total_plays
0,49,689
1,80,665
2,97,557
3,15,463
4,44,397


Most played songs

In [22]:
run_query("""
SELECT s.title, COUNT(*) AS play_count
FROM songplays sp
JOIN songs s ON sp.song_id = s.song_id
GROUP BY s.title
ORDER BY play_count DESC
LIMIT 10;
""")

,title,play_count
0,Setanta matins,1


Plays per day

In [23]:
run_query("""
SELECT DATE(start_time) AS play_date, COUNT(*) AS total_plays
FROM songplays
GROUP BY play_date
ORDER BY play_date;
""")


,play_date,total_plays
0,2018-11-01,11
1,2018-11-02,155
2,2018-11-03,100
3,2018-11-04,144
4,2018-11-05,356
5,2018-11-06,154
6,2018-11-07,174
7,2018-11-08,161
8,2018-11-09,252
9,2018-11-10,87
